In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# 이미지에 들어가는 한글을 제대로 보기 위해 한글 폰트 적용
import platform
font_dict = {
    'Linux': 'Noto Sans CJK KR',
    'Darwin': 'Apple SD Gothic Neo', # macOS
    'Windows': 'Malgun Gothic' # MS-Windows
}
try:
    mpl.rc('font', family=font_dict[platform.system()])
except:
    pass
mpl.rc('axes', unicode_minus=False)

%matplotlib inline

# [All about 따릉이 EDA, 2편] 따릉이, 이용량은 얼마나 변했을까? by 흠시

**출처:** https://dailyheumsi.tistory.com/87, https://github.com/heumsi/Seoul-Public-bicycles-EDA

**데이터:** 서울특별시 공공자전거 대여소별 이용정보(월별) @[서울 열린데이터 광장](https://data.seoul.go.kr)
  - 대여소별 대여, 반납 정보, 대여소번호, 대여소명, 대여건수, 반납건수 정보를 제공
  - https://data.seoul.go.kr/dataList/OA-15249/F/1/datasetView.do
  - `공공자전거 대여소별 이용정보_201912_202005.xlsx` (용량 0.5MB)
  - `공공자전거 대여소별 이용정보_201906_201911.xlsx` (용량 0.5MB, 수정일 2019.12.23)
  - `공공자전거 대여소별 이용정보_201812_201905.xlsx` (용량 0.5MB, 수정일 2019.06.18)

> [흠시] 이번에 알아보려고 하는 것은, **2015년 9월 - 2018년 11월까지의 따릉이 사용량**에 대한 이야기다.  
즉, 이전에 없던 **시간의 개념**이 분석에 더해진다.  
년도, 월에 따라 사용량이 어떻게 달라지는지. 각 지역별로 차이가 있는지 등, **시간에 따라 사용량이 어떻게 달라지는지,** 세세히 살펴보자.  
여기서 사용량은 대여량 + 반납량을 말한다.

In [ ]:
from pathlib import Path

공공자전거_대여소별_이용정보_월별 = Path('../data/서울특별시 공공자전거 대여소별 이용정보(월별)')

공공자전거_대여소별_이용정보_201812_201905 = \
    공공자전거_대여소별_이용정보_월별 / '공공자전거 대여소별 이용정보_201812_201905.xlsx'
공공자전거_대여소별_이용정보_201812_201905

In [ ]:
# 아무런 옵션을 주지 않으면 엑셀 파일의 첫 번째 (대여)시트를 읽는다.

df1 = pd.read_excel(공공자전거_대여소별_이용정보_201812_201905)
df1.head()

In [ ]:
# 대여일자 컬럼을 datetime 형식으로 읽으려면?

df1 = pd.read_excel(공공자전거_대여소별_이용정보_201812_201905,
    parse_dates=['대여일자'],
    date_parser=lambda s: s+'01'
)
df1.head()

### 결측치 확인

In [ ]:
import missingno as msno

msno.matrix(df1);

In [ ]:
df1.info()

In [ ]:
# 데이터 결측치가 있는 로우(row)는 어디?

df1[df1.isna().any(axis=1)]

In [ ]:
df1 = df1.dropna()
df1.info()

In [ ]:
# 대여건수 컬럼을 정수(int64)로 읽으려면?

df1 = df1.astype({'대여건수': 'int64'})
df1.info()

In [ ]:
# 이번에는 같은 방법으로 두 번째 (반납)시트(sheet)를 읽어들이자.

df2 = pd.read_excel(공공자전거_대여소별_이용정보_201812_201905,
    sheet_name='반납',
    parse_dates=['반납일자'],
    date_parser=lambda s: s+'01'
).dropna().astype({'반납건수': 'int64'})
df2.info()

In [ ]:
# 지역구와 대여소명이 같고 대여일자와 반납일자가 동일하도록 두 개의 데이터 프레임을 합해보자.

df = df1.merge(df2,
    left_on=['구분', '대여소명', '대여일자'],
    right_on=['구분', '대여소명', '반납일자']
)
df.info()

In [ ]:
# 같은 작업을 2019년 6월부터 2019년 11월까지 데이터에도 적용하자.
# 컬럼 이름이 바뀌었음에 주의하자.

공공자전거_대여소별_이용정보_201906_201911 = \
    공공자전거_대여소별_이용정보_월별 / '공공자전거 대여소별 이용정보_201906_201911.xlsx'

df1 = pd.read_excel(공공자전거_대여소별_이용정보_201906_201911,
    parse_dates=['대여 일자 / 월'],
    date_parser=lambda s: s+'01'
).rename(
    columns={'대여소 그룹': '구분',
             '대여소 명': '대여소명',
             '대여 일자 / 월': '대여일자',
             '대여 건수': '대여건수'}
).dropna().astype({'대여건수': 'int64'})
df1.info()

In [ ]:
# 동일한 로우(row)가 있는지 살펴보자.

df1[df1.duplicated(keep=False)]

In [ ]:
# 동일한 로우를 모두 지우자.

df1 = df1.drop_duplicates(keep=False)
df1.info()

In [ ]:
# 같은 방법으로 두 번째 (반납)시트(sheet)를 읽어들이자.
# 이번에는 '그룹명 없음'과 '대여소명 없음'을 전부 결측치로 가정한다.

df2 = pd.read_excel(공공자전거_대여소별_이용정보_201906_201911,
    sheet_name='반납',
    parse_dates=['반납 일자 / 월'],
    date_parser=lambda s: s+'01',
    na_values=['그룹명 없음', '대여소 명 없음']
).rename(
    columns={'대여소 그룹': '구분',
             '대여소 명': '대여소명',
             '반납 일자 / 월': '반납일자',
             '반납 건수': '반납건수'}
).dropna().astype({'반납건수': 'int64'})
df2.info()

In [ ]:
# 지역구와 대여소명이 같고 대여일자와 반납일자가 동일하도록 두 개의 데이터 프레임을 합해보자.

_df = df1.merge(df2,
    left_on=['구분', '대여소명', '대여일자'],
    right_on=['구분', '대여소명', '반납일자']
)
_df.info()

In [ ]:
# 2018년 12월부터 2019년 5월까지 데이터(df)와 2019년 6월부터 2019년 11월까지 데이터(_df)를 붙이자.

df = pd.concat([df, _df], ignore_index=True)
df.info()

In [ ]:
# 같은 작업을 2019년 12월부터 2020년 5월까지 데이터에도 적용하자.

공공자전거_대여소별_이용정보_201912_202005 = \
    공공자전거_대여소별_이용정보_월별 / '공공자전거 대여소별 이용정보_201912_202005.xlsx'

df1 = pd.read_excel(공공자전거_대여소별_이용정보_201912_202005,
    parse_dates=['대여 일자 / 월'],
    date_parser=lambda s: s+'01',
    na_values=['그룹명 없음', '대여소 명 없음']
).rename(
    columns={'대여소 그룹': '구분',
             '대여소 명': '대여소명',
             '대여 일자 / 월': '대여일자',
             '대여 건수': '대여건수'}
).dropna().astype({'대여건수': 'int64'})
df1.info()

In [ ]:
# 같은 방법으로 두 번째 (반납)시트(sheet)를 읽어들이자.

df2 = pd.read_excel(공공자전거_대여소별_이용정보_201912_202005,
    sheet_name='반납',
    parse_dates=['반납 일자 / 월'],
    date_parser=lambda s: s+'01',
    na_values=['그룹명 없음', '대여소 명 없음']
).rename(
    columns={'대여소 그룹': '구분',
             '대여소 명': '대여소명',
             '반납 일자 / 월': '반납일자',
             '반납 건수': '반납건수'}
).dropna().astype({'반납건수': 'int64'})
df2.info()

In [ ]:
# 지역구와 대여소명이 같고 대여일자와 반납일자가 동일하도록 두 개의 데이터 프레임을 합해보자.

_df = df1.merge(df2,
    left_on=['구분', '대여소명', '대여일자'],
    right_on=['구분', '대여소명', '반납일자']
)
_df.info()

In [ ]:
# 2018년 12월부터 2019년 11월까지 데이터(df)와 2019년 12월부터 2020년 5월까지 데이터(_df)를 붙이자.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

df = pd.concat([df, _df], ignore_index=True)
df.info()

In [ ]:
# 연과 월 기준으로 데이터를 처리할 예정이니 연(year)과 월(month) 컬럼을 추가하자.

df['년'] = df['대여일자'].dt.year
df['월'] = df['대여일자'].dt.month

In [ ]:
# 데이터 처리가 용이하도록 구분과 대여소명 컬럼을 파이썬 오브젝트(string) 형식에서 category 형식으로 바꾸자.
# https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html

df['구분'] = df['구분'].astype('category')
df['대여소명'] = df['대여소명'].astype('category')
df.info()

---
## 1. 연, 월 기준 사용량은 어떨까?

> [흠시] 먼저 연, 월 기준으로 막대그래프를 그려보자.

In [ ]:
year = df.groupby('년')[['대여건수', '반납건수']].sum()
year

In [ ]:
ax = year.plot.bar(
    stacked=True,
    rot=0,
    title="연 기준 사용량",
    figsize=(10, 4)
)
ax.set_frame_on(False)
#ax.ticklabel_format(axis='y', style='plain')
ax.legend(loc='upper left', frameon=False);

> [흠시] **확실히 해가 지날수록, 사용량이 많이 늘고있음**을 알 수 있다.  
다음으로 월 기준으로 보면,

In [ ]:
month = df.groupby(['년', '월'])[['대여건수', '반납건수']].sum()
month

In [ ]:
ax = month.plot.bar(
    stacked=True,
    rot=45,
    title="월 기준 사용량",
    figsize=(10, 4)
)
ax.set_frame_on(False)
#ax.ticklabel_format(axis='y', style='plain')
ax.legend(frameon=False);

> [흠시] **여름 전후인, 5월 그리고 9, 10월에 피크**를 찍고, 가장 **사용량이 많은 달은 9월**이다.  
추운 겨울과, 더운 여름에는 사용량이 줄어드는 듯 보인다.  
무엇보다, **1년을 주기로 이러한 패턴**을 보인다. 즉, 데이터 패턴을 볼 때, 기본적으로 1년 단위로 봐야한다.  

> [흠시] 그럼 해가 지나면서 얼마나 늘었을까?  
가장 피크인 달인 9월을 기준으로 직전 해 대비 증가율을 살펴보자.

In [ ]:
may = month.loc[[(2019, 5), (2020, 5)]]['대여건수']
may

In [ ]:
a = may.to_numpy()
increasing_ratio = pd.Series((a[1:] - a[:-1]) / a[:-1] * 100, index=may[1:].index.get_level_values(0))
increasing_ratio

In [ ]:
ax = increasing_ratio.plot.bar(
    rot=0,
    title="5월 사용량, 직전 해 대비 증가율",
    color='grey'
)
ax.set_frame_on(False)
for p in ax.patches:
    x, y, width, height = p.get_bbox().bounds 
    ax.annotate(f"{height:.2f}%", (x+width/2, height), ha='center') # ha = horizontalalignment

> [흠시] 2016년에는 2015년 대비 2615% 나 증가했다가, 그 이후로는 직전 해 대비 증가율이 확 감소하긴 했지만,  
작년 대비 사용량은 꾸준히 증가했으므로, 절대적인 값은 더 클 것이다. (굳이 살펴보진 않겠다.)

---
## 2. 지역구별로 사용량은 다를까?

> [흠시] 전체 기간, 즉 15년 9월 - 18년 11월 동안의 각 지역별 사용량을 살펴보자.

In [ ]:
sum_of_use = (df.groupby('구분')[['대여건수', '반납건수']]
    .sum()
    .sort_values(by='대여건수', ascending=False)
)
sum_of_use.head()

In [ ]:
# 컬럼들의 상관관계를 살펴보자.

sum_of_use.corr()

In [ ]:
ax = sum_of_use.plot.bar(
    rot=0,
    xlabel="",
    title="지역구별 따릉이 사용량",
    figsize=(15, 4)
)
ax.set_frame_on(False)
# ax.ticklabel_format(axis='y', style='plain')
ax.legend(frameon=False);

In [ ]:
import json, folium

geo_path = Path('../data/seoul_municipalities_geo_simple.json')
with open(geo_path, encoding='utf-8') as fp:
    geo_str = json.load(fp)
    
bike_map = folium.Map(
    location=[37.541, 126.986],
    zoom_start=10.8,
    tiles='CartoDB positron'
)
folium.Choropleth(geo_str,
    data=sum_of_use['대여건수'],
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='Blues'
).add_to(bike_map)
bike_map

> [흠시]  
>  - 대여량과 반납량은 비슷하다. (대여하면 반납하니 당연...)
>  - 마포와, 영등포의 사용량이 압도적으로 높다. 
>  - 제일 낮은 지역은 강북, 금천, 도봉이다.
>  - 제일 낮은 사용량과 제일 큰 사용량의 차이가 대략 16배 정도 난다. 너무 큰 데.. 이상하다.  

> [흠시] 이전 포스트에서 2018년 11월 기준, 대여소가 가장 많은 곳은 강남 3구였다.  
**대여소가 많으면 일반적으로 사용량도 높을 것 같은데 왜 강남 3구 사용량은 별로 안높을까?**

> [흠시] 여기서 이런 의문이 든다.  
>> 마포와 영등포에 가장 먼저 대여소들이 설치되고, 다른 지역은 뒤늦게 설치되어서, 자연스레 먼저 이용되고 있던 마포와 영등포만 너무 높은게 아닐까?

> [흠시] 이를 확인해보기 위해, 잠깐 샛길로 빠져나와, 시간에 따른 각 지역구의 대여소 개수를 살펴보자.

### 2.1. 시간에 따른 각 지역구의 대여소 개수 살펴보기

![](reshaping_pivot.png)
- **출처:** https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#pivot-tables

In [ ]:
# 동일한 인덱스와 동일한 컬럼을 가지는 로우들이 여럿 있을 때, 이러한 값들에 대해 sum, mean 등의
# aggregation 함수를 적용할 수 있는 명령이 바로 pivot_table 이다.
# pivot_table의 aggfunc 디폴트 값은 numpy.mean이다.

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nunique.html

pvt_num_rental = df.pivot_table(
    index='구분',
    columns=['년', '월'],
    values='대여소명',
    aggfunc='nunique'
).fillna(0).drop(
    columns=[(2018, d) for d in range(1, 12)] + [(2020, d) for d in range(6, 13)]
)
pvt_num_rental.head()

In [ ]:
pvt_num_rental.sort_values(
    by=[(2020, 5), (2020, 4)],
    ascending=False,
    inplace=True
)
pvt_num_rental.head()

In [ ]:
ax = pvt_num_rental.T.plot.line(
    style='.-',
    title="지역별 년, 월 기준 대여소 개수",
    figsize=(15, 10)
)
ax.legend(loc='upper left')
ax.set_frame_on(False)

> [흠시] 데이터 전 기간동안의 각 지역별 대여소 개수를 히트맵으로 시각화 해보자.

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.heatmap.html

_, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(pvt_num_rental,
    square=True,
    annot=True,
    fmt="d",
    cmap='Blues',
    linewidths=0.5,
    cbar=False,
    ax=ax
)
ax.set_title("지역별 년, 월 기준 대여소 개수");

> [흠시] 정리해보면,  
> - 예상대로, 2015.10 이전부터, **영등포, 마포, 서대문**을 중심으로 대여소가 **선행적으로 설치**되었다.
> - 2015.10 에 종로, 중구, 성동, 광진구에 일부 대여소가 처음 설치되었고, 마포는 이전량의 2배를 늘어났다.
> - 2016.7, 2017.5 을 기점으로, 기존 지역 및 새로운 지역들에 대여소가 설치됐다.
> - 그 이후로, 계속해서 대여소는 늘어나고, 특히 **강남, 서초, 송파에 집중적으로 대여소들이 추가**되었다.

> [흠시] 즉, **강남 3구의 경우, 가장 많이 설치됐지만 가장 최근에 대량으로 설치되었기 때문에**, 대여소는 많지만 사용량이 적었던 것이다.

> [흠시] 한편, 서울시는 전 지역에 골고루 대여소를 설치한게 아니라,   
**특정 시기에, 특정 지역에 집중적으로 대여소를 설치하는 방식으로 대여소를 늘려왔음을 알 수 있다.**  
이를 확인하기 위해, 시간에따른 전체 대여소 개수를 막대 그래프로 살펴보면, 아래와 같다.

In [ ]:
# 월별 대여소 개수

ax = pvt_num_rental.sum(axis=0).plot.bar(
    rot=45,
    title="년, 월 기준 대여소 개수",
    figsize=(10, 4)
)
ax.set_frame_on(False)

In [ ]:
# 연별 대여소 개수

ax = pvt_num_rental.sum(axis=1, level=0).sum(axis=0).plot.bar(
    rot=0,
    figsize=(10, 4)
)
for p in ax.patches:
    x, y, width, height = p.get_bbox().bounds 
    ax.annotate(f"{height:.0f}", (x+width/2, height), ha='center') #ha = horizontalalignment
ax.set_frame_on(False)

### 2.2. 시간에 따른 각 지역구의 사용량 살펴보기

> [흠시] 먼저 러프하게, 선 그래프로 사용량 추세를 살펴보자.

In [ ]:
# 지역구 명 인덱스

region_index = pvt_num_rental.index
region_index

In [ ]:
# 대여건수

pvt_num_use = df.pivot_table(
    index='구분',
    columns=['년', '월'],
    values='대여건수',
    aggfunc='sum'
).fillna(0)
pvt_num_use = pvt_num_use.astype('int64')
pvt_num_use = pvt_num_use.reindex(index=region_index)
pvt_num_use.head()

In [ ]:
ax = pvt_num_use.T.plot.line(
    style='.-',
    title="년, 월 기준 각 지역별 대여건수",
    figsize=(15, 10)
)
ax.set_frame_on(False)
ax.legend(frameon=False);

> [흠시] **어느 지역구도 예외 없이**, 일년 동안의 사용량 추세 패턴(여름 전후로 사용량이 제일 많은)이 같다.    
이를 좀 더 수치적으로 편하게 살펴보기 위해, 히트맵으로 시각화해서 보면,

In [ ]:
unit = 1000

_, ax = plt.subplots(figsize=(15, 15))
sns.heatmap((pvt_num_use/unit).astype('int64'),
    square=True,
    annot=True,
    fmt="d",
    cmap='Greens',
    linewidths=0.5,
    ax=ax
)
ax.set_title("년, 월 기준 각 지역별 대여건수 (천 단위)");

> [흠시]  
> - 먼저, 0으로 값이 채워진 칸은, 당시에 대여소 설치가 아직 안됬기 때문이다.
> - 어김 없이, 모든 지역에서 해가 지날수록 사용량이 많아진다.
> - **영등포, 마포는 초기 때부터, 꾸준하게 가장 사용량이 많은 지역**이다.
> - 18년 11월 기준, 대여소 개수가 가장 많던**강남 3구 중, 눈에 띄게 사용량이 많은 지역은 송파**이다.

> [흠시] 반납건수도 대여건수와 별 다르지 않다.

In [ ]:
# 반납건수

pvt_num_return = df.pivot_table(
    index='구분',
    columns=['년', '월'],
    values='반납건수',
    aggfunc='sum'
).fillna(0)
pvt_num_return = pvt_num_return.astype('int')
pvt_num_return = pvt_num_return.reindex(index=region_index)
pvt_num_return.head()

In [ ]:
ax = pvt_num_return.T.plot.line(
    style='.-',
    title="년, 월 기준 각 지역별 반납건수",
    figsize=(15, 10)
)
ax.set_frame_on(False)
ax.legend(frameon=False);

In [ ]:
unit = 1000

_, ax = plt.subplots(figsize=(15, 15))
sns.heatmap((pvt_num_return/unit).astype('int64'),
    square=True,
    annot=True,
    fmt="d",
    cmap='Oranges',
    linewidths=0.5,
    cbar=False,
    ax=ax
)
ax.set_title("년, 월 기준 각 지역별 반납건수 (천 단위)");

> [흠시] 하지만, 위에서는 각 <u>지역별 대여소 개수를 고려하지 않고 사용량을 산출</u>해냈기 때문에,   
지역간 사용량을 비교할 때, 이 값이 공정해보지 않는다.  
따라서, 지역간 좀 더 세밀하게 비교해보기 위해, 이번엔 지역별 대여소 개수까지도 고려한 값을 산출해보자.

### 2.3. 지역구별 대여소당 평균 이용량

> [흠시] 좀 더 공정한 비교를 위해, 지역구별 대여소 개수까지 고려한, **지역의 대여소 평균 이용량**이라는 개념을 도입해보자.  
`지역의 대여소 평균 이용량 = 지역의 대여건수의 합 / 지역의 대여소 개수`로 정의된다.  
즉, 한 대여소당, 평균 이용량이 얼마나 되는지, 지역구별로 살펴보자.

In [ ]:
pvt_use_ratio = (pvt_num_use / pvt_num_rental).fillna(0)
pvt_use_ratio = pvt_use_ratio.astype('int64')
pvt_use_ratio = pvt_use_ratio.reindex(index=region_index)
pvt_use_ratio.head()

In [ ]:
_, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(pvt_use_ratio.astype('int64'),
    square=True,
    annot=True,
    fmt="d",
    cmap='Blues',
    linewidths=0.5,
    cbar=False,
    ax=ax
)
ax.set_title("지역구별 대여소당 평균 이용량")
ax.set_ylabel("");

> [흠시]  
> - 역시나, **마포, 영등포는 대여소당 평균 이용률도 가장 높은 편**이다.
> - 새롭게 눈에 띄는 것은 **광진구**다. 전체 이용량이 높은 것은 아니었지만,   
    **자전거 대여소 대비 이용량이 많은 편이다. 자전거 대여소를 늘려주면, 이용량 자체가 더 늘어날까?**
> - 대여소가 많았던 **강남 3구중, 송파를 제외한 다른 지역은 왜 이용률이 높지 않을까?**  
    대여소가 많이 설치되어도, 기대만큼 사람들이 잘 이용하지 않는걸까?  
    아니면 비교적 최근에 설치되어, 아직 사람들이 잘 모르는 걸까?
    
> [흠시] 전체 기간은 아직 대여소가 설치 안된 시점을 포함하므로,   
**어느정도 대여소가 설치된 17년 11월 - 18년 11월의 데이터**만 가지고, 각 지역의 대여소당 평균 이용률의 합을 비교해보자.

In [ ]:
sum_use_ratio = pvt_use_ratio.sum(axis=1)
sum_use_ratio.sort_values(ascending=False, inplace=True)

ax = sum_use_ratio.plot.bar(
    rot=0,
#    color='C0',
    xlabel="",
    title="지역구별 대여소당 평균 이용량의 합 (18년 12월 - 20년 5월)",
    figsize=(15, 5)
)
ax.set_frame_on(False)

> [흠시]  
> - 위에서 살펴본 것과 마찬가지로, **영등포, 마포, 광진구**가 제일 높다.
> - 대여소당 평균 이용량이 가장 높은 지역(마포)과 낮은 지역(금천) 간의 차이가 **약 4배** 가량 난다.

---
## 정리  

1. 따릉이 이용률은 계속해서 증가하고 있지만, 증가율 폭은 이전 대비 감소하고 있다.
2. 따릉이 이용이 **가장 꾸준하게 잘 사용되었던 지역은 영등포, 마포**이다. 대여소당 평균 이용률도 높고, 주목해서 봐야할 지역이다.      
3. **따릉이 대여소는 초기에 특정 월에 몰아서 설치되었지만, 17년도 부터는 꾸준하게 설치**되었다.
4. **광진은 대여소당 특히 평균 이용률이 높은 편**이다.
5. **송파는 비교적 최근에 이용량**이 눈에띄게 높았다.

---
## 남아있는 질문들

### 질문 1) 해당 지역에 자전거 대여소가 많아지면, 전체 이용률도 올라갈까?

전체 이용률이.. 자전거 대여소 수에만 영향받는게 아니기 때문에. 자전거 대여소 수 증가 -> 전체 이용률 수 증가 인지 알 수 없을듯 하다.

### 질문 2) 일일 정류소당 대여건수를 예측해볼 수 있을까?

해볼만 한 것 같다.  
Kaggle Competition 에 꽤 유명한 "Predicting Bike sharing demand" 를 참고할 수도 있고, 일일 날씨 데이터와 조인하면, 시도해봄직 하다.  
추후 해볼만한 시도다.